In [17]:
import pandas as pd
import os

In [18]:
os.getcwd()

'/home/shiwam/ceu-ds-project-groupB-2026/Gret tit'

In [23]:
path_to_folder = '/mnt/c/Users/user/Downloads/great tit'
file_path = os.path.join(path_to_folder,'gretit_cleaned_europe_only.csv')
print(file_path)

/mnt/c/Users/user/Downloads/great tit/gretit_cleaned_europe_only.csv


In [26]:
# We use sep='\t' because eBird files are Tab-Separated
# We use nrows=5 to just grab a tiny sample
try:
    df = pd.read_csv(file_path, sep=',')

    # This makes the output "beautiful" in the console
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', 1000)        # Don't wrap lines
    
    print("---eBird Data Preview---")
    print(df.head(20))
    
    print("\n--- Column Names (So you know what you can filter by) ---")
    print(df.columns.tolist())

except FileNotFoundError:
    print("Error: The file wasn't found. Double-check the file path!")

---eBird Data Preview---
   SCIENTIFIC NAME  OBSERVATION COUNT COUNTRY CODE                                           LOCALITY   LATITUDE  LONGITUDE OBSERVATION DATE SAMPLING EVENT IDENTIFIER OBSERVATION TYPE
0      Parus major                3.0           AD                                Canillo Ski Station  42.555799   1.617608       2018-04-05                 S44289207       Incidental
1      Parus major                1.0           AD                       by S. side of CG2 in Canillo  42.568000   1.603400       2018-06-11                 S59124583       Incidental
2      Parus major                1.0           AD                                Canillo Ski Station  42.555799   1.617608       2019-11-17                 S61522924        Traveling
3      Parus major                1.0           AD                                      Vall d'Incles  42.585705   1.664708       2021-05-01                S275308560       Historical
4      Parus major                1.0           AD     

In [27]:
df.shape

(1360446, 9)

In [28]:
df.dtypes

SCIENTIFIC NAME                  str
OBSERVATION COUNT            float64
COUNTRY CODE                     str
LOCALITY                         str
LATITUDE                     float64
LONGITUDE                    float64
OBSERVATION DATE                 str
SAMPLING EVENT IDENTIFIER        str
OBSERVATION TYPE                 str
dtype: object

Converting the date in pandas object datetime


In [29]:
# Convert the 'strings' into actual date objects
df['OBSERVATION DATE'] = pd.to_datetime(df['OBSERVATION DATE'])

In [35]:
# Create helper columns for your year-by-year analysis
df['YEAR'] = df['OBSERVATION DATE'].dt.year
df['MONTH'] = df['OBSERVATION DATE'].dt.month

In [36]:
df.head()

,SCIENTIFIC NAME,OBSERVATION COUNT,COUNTRY CODE,LOCALITY,LATITUDE,LONGITUDE,OBSERVATION DATE,SAMPLING EVENT IDENTIFIER,OBSERVATION TYPE,YEAR,MONTH
0,Parus major,3.0,AD,Canillo Ski Station,42.555799,1.617608,2018-04-05,S44289207,Incidental,2018,4
1,Parus major,1.0,AD,by S. side of CG2 in Canillo,42.568000,1.603400,2018-06-11,S59124583,Incidental,2018,6
2,Parus major,1.0,AD,Canillo Ski Station,42.555799,1.617608,2019-11-17,S61522924,Traveling,2019,11
3,Parus major,1.0,AD,Vall d'Incles,42.585705,1.664708,2021-05-01,S275308560,Historical,2021,5
4,Parus major,1.0,AD,Vall d'Incles,42.585705,1.664708,2021-05-01,S215685554,Historical,2021,5


In [38]:
# Sort from oldest to newest sightings
df = df.sort_values(by=['OBSERVATION DATE']).reset_index(drop=True)

In [39]:
df.head()

,SCIENTIFIC NAME,OBSERVATION COUNT,COUNTRY CODE,LOCALITY,LATITUDE,LONGITUDE,OBSERVATION DATE,SAMPLING EVENT IDENTIFIER,OBSERVATION TYPE,YEAR,MONTH
0,Parus major,1.0,NO,Flekkerøy--Berge,58.075198,7.978821,2018-01-01,S42155305,Historical,2018,1
1,Parus major,1.0,SE,Gärdet,59.347822,18.096542,2018-01-01,S41624313,Stationary,2018,1
2,Parus major,5.0,SE,Matningen S Säbysjön,59.423084,17.872342,2018-01-01,S47493977,Incidental,2018,1
3,Parus major,6.0,GB,Walcot,52.703139,-2.604058,2018-01-01,S53881074,Stationary,2018,1
4,Parus major,1.0,SE,Gärdet,59.347822,18.096542,2018-01-01,S41492070,Stationary,2018,1


We can now analyze them by seasons because of months and obviously with diffrent years


In [ ]:
import folium
from folium.plugins import HeatMap


# Create a folder to store your maps
if not os.path.exists('migration_maps'):
    os.makedirs('migration_maps')

target_year = 2021

# Loop through the spring migration months (March to June)
for month in range(3, 7):
    # 1. Filter the main 'df' for the specific month and year
    df_month = df[(df['YEAR'] == target_year) & (df['MONTH'] == month)]
    
    if df_month.empty:
        print(f"No data found for month {month}, skipping...")
        continue

    # 2. Initialize the map centered on Europe
    m = folium.Map(location=[50, 10], zoom_start=4, tiles='cartodbpositron')

    # 3. Prepare coordinates and weights (Observation Count)
    # We use .values.tolist() for high-performance data injection into Folium
    heat_data = df_month[['LATITUDE', 'LONGITUDE', 'OBSERVATION COUNT']].values.tolist()

    # 4. Add the HeatMap layer
    # Adjust radius to make the "glow" larger or smaller based on density
    HeatMap(heat_data, radius=15, blur=10, min_opacity=0.5).add_to(m)

    # 5. Save the map
    file_name = f'migration_maps/great_tit_{target_year}_{month:02d}.html'
    m.save(file_name)
    print(f"Generated map: {file_name} ({len(df_month)} sightings)")

Generated map: migration_maps/great_tit_2021_03.html (42230 sightings)
Generated map: migration_maps/great_tit_2021_04.html (48701 sightings)
Generated map: migration_maps/great_tit_2021_05.html (40240 sightings)
Generated map: migration_maps/great_tit_2021_06.html (20603 sightings)


In [43]:
# 1. Calculate the average Latitude and Longitude for each month
# This finds the "geographic center" of the Great Tit population per month
trajectory_df = df.groupby(['YEAR', 'MONTH'])[['LATITUDE', 'LONGITUDE']].mean().reset_index()

# 2. Filter for a specific year to see that year's path
path_2021 = trajectory_df[trajectory_df['YEAR'] == 2021]

In [44]:
trajectory_df

,YEAR,MONTH,LATITUDE,LONGITUDE
0,2018,1,47.855535,3.445401
1,2018,2,47.318937,3.114228
2,2018,3,47.608192,3.799604
3,2018,4,47.430774,3.548104
4,2018,5,47.387714,4.041931
5,2018,6,48.187791,4.102649
6,2018,7,48.266082,4.306750
7,2018,8,48.437561,4.194949
8,2018,9,47.887479,4.192645
9,2018,10,48.165053,4.831230


In [45]:
import folium

# Create the map
m_path = folium.Map(location=[50, 10], zoom_start=4, tiles='cartodbpositron')

# Create a list of coordinates for the line (Jan -> Feb -> March...)
points = path_2021[['LATITUDE', 'LONGITUDE']].values.tolist()

# Add the trajectory line
folium.PolyLine(points, color="blue", weight=2.5, opacity=0.8).add_to(m_path)

# Add markers for each month so you can see the direction
for i, row in path_2021.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=5,
        popup=f"Month: {int(row['MONTH'])}",
        color='red',
        fill=True
    ).add_to(m_path)

m_path.save('great_tit_trajectory_2021.html')